In [1]:
import os
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import types
import torch
import utils
import TextNet
import TeacherNet
import ImageNet
import torch.optim as optim
import time
import sys
import numpy as np
from torch.autograd import Variable
from embeddings import build_vocab, build_embed, build_train, replace_values
from transformers import DistilBertTokenizer
import torchtext
from torchtext.vocab import GloVe


from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler


Torch Version:  1.3.0+cu92


In [2]:
train_img = torch.load("../cached_data/train_img")
train_cap = torch.load("../cached_data/train_cap")
train_mask = torch.load("../cached_data/train_mask")

val_img = torch.load("../cached_data/val_img")
val_cap = torch.load("../cached_data/val_cap")
val_mask = torch.load("../cached_data/val_mask")

print("Loaded train data", train_img.size(), train_cap.size(), train_mask.size())
print("Loaded val data", val_img.size(), val_cap.size(), val_mask.size())

Loaded train data torch.Size([10000, 3, 224, 224]) torch.Size([10000, 52]) torch.Size([10000, 52])
Loaded val data torch.Size([5000, 3, 224, 224]) torch.Size([5000, 43]) torch.Size([5000, 43])


In [3]:
DELTA = 0.1
BATCH_SIZE = 8
NB_EPOCHS = 10
LOGGER = utils.Logger()
EMB_CACHE = os.path.expanduser("../../words/glove/")

# VOCAB_SIZE = len(train_mask)
# EMBED_DIM = train_cap.size()[1]
# NUN_CLASS = len(train_cap)
# model = TextNet.Text_Net(VOCAB_SIZE, EMBED_DIM, NUN_CLASS)


In [4]:
# train_data = TensorDataset(train_img, train_cap, train_mask)
# train_sampler = RandomSampler(train_data)
# train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=BATCH_SIZE, num_workers=2)

# valid_data = TensorDataset(val_img, val_cap, val_mask)
# valid_sampler = SequentialSampler(valid_data)
# valid_dataloader = DataLoader(valid_data, sampler=valid_sampler, batch_size=BATCH_SIZE * 2, num_workers=2)


In [5]:
vocab = build_vocab(train_cap) # all unique words
print(vocab[0])


embedding_matrix = build_embed(vocab) # embeddings gloves
print(np.shape(embedding_matrix))

device = torch.device("cuda:1")
text_net = TextNet.Text_Net(embedding_matrix )
text_net.to(device)
#text_net = TextNet.Text_Net(VOCAB_SIZE, EMBED_DIM, NUN_CLASS)

closeup
(5838, 300)


Text_Net(
  (embedding): Embedding(5838, 300)
  (conv1): Conv1d(52, 300, kernel_size=(3,), stride=(1,))
  (conv2): Conv1d(300, 300, kernel_size=(3,), stride=(1,))
  (conv3): Conv1d(300, 300, kernel_size=(3,), stride=(1,))
  (fc1): Linear(in_features=21900, out_features=9216, bias=True)
)

In [6]:
# print(train_cap.size())
# print(len(train_cap[0]))
# print(train_cap[0])
# tokenizer = DistilBertTokenizer.from_pretrained("distilbert-base-uncased")
# words = tokenizer.decode(train_cap[0].numpy()).split(' ')
# print(words)
# print(words[0] in vocab)
# print(len(words))

In [7]:
# train = replace_values(train_cap, vocab)
# torch.save(train, "../cached_data/cap_replace")


train_cap = torch.load("../cached_data/cap_replace")
train_cap[0]
tokenizer = DistilBertTokenizer.from_pretrained("distilbert-base-uncased")
words = tokenizer.decode(train_cap[0].numpy()).split(' ')

In [8]:
print(train_cap[0])
print(len(train_cap[10]))

tensor([5834,    0,    1,    2,    1,    3,    4,    5,    6,    7,    8, 5836,
        5835, 5835, 5835, 5835, 5835, 5835, 5835, 5835, 5835, 5835, 5835, 5835,
        5835, 5835, 5835, 5835, 5835, 5835, 5835, 5835, 5835, 5835, 5835, 5835,
        5835, 5835, 5835, 5835, 5835, 5835, 5835, 5835, 5835, 5835, 5835, 5835,
        5835, 5835, 5835, 5835])
52


In [9]:
# print(train_cap.size())
# print(torch.max(train_cap,0))

In [10]:
train_data = TensorDataset(train_img, train_cap, train_mask)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=BATCH_SIZE, num_workers=2)

In [11]:
# TEXT = torchtext.data.Field(sequential=True, lower=True, include_lengths=True, batch_first=True, tokenize='spacy')
# TEXT.build_vocab(train, vectors=GloVe(name='6B', dim=300, cache=EMB_CACHE))
# train = TEXT.vocab
# print(train.vectors.size())
# print(len(train))

In [12]:
model_name = "alexnet"
feature_extract = True
vision_net = ImageNet.Image_Net()
vision_net = vision_net.initialize_model(model_name, feature_extract, use_pretrained=True)
vision_net.to(device)


teacher_net = TeacherNet.Teacher_Net()
ranking_loss = TeacherNet.RankingLossFunc(DELTA)
teacher_net.to(device)
ranking_loss.to(device)

# print(text_net)
# print(vision_net)

# embeds = Variable(torch.Tensor((BATCH_SIZE, 16, 300)), requires_grad=True)

4096


RankingLossFunc()

In [13]:
# optimizer
params_to_update_share = []
params_to_update_img = vision_net.parameters()
params_to_update_txt = []


params_to_update = list(params_to_update_share) + list(params_to_update_img) + list(params_to_update_txt)
optimizer = optim.Adam(params_to_update, lr=0.0001)


print("Start to train")
start_time = time.time()
for epoch in range(NB_EPOCHS):
    running_loss = 0.0
    running_corrects = 0.0
    total_samples = 0

    for step, batch in enumerate(train_dataloader):
        img, cap, mask = tuple(t.to(device, dtype=torch.float) for t in batch)
        cap = torch.tensor(cap).to(device,torch.int64)
#         print(img.size())
#         print(cap.size())
        
        with torch.set_grad_enabled(False):
            img_feature = vision_net.forward(img)
#             print(img_feature.size()) 
#             cap = np.expand_dims(cap.cpu(), axis=2)
#             cap=torch.tensor(cap)
#             print(cap.size(), cap.type)
            txt_feature = text_net.forward(cap)
            
        with torch.set_grad_enabled(True):
            img_vec = teacher_net.forward(img_feature)
            txt_vec = teacher_net.forward(txt_feature)
    
            loss = ranking_loss(img_vec, txt_vec)
            preds = teacher_net.predict(img_vec, txt_vec)
            optimizer.step()
            optimizer.zero_grad()
    
        running_loss += loss.item() * BATCH_SIZE
        running_corrects += sum([(i == preds[i]) for i in range(len(preds))])
        total_samples += len(preds)
        #print(running_loss)
        
    print("Epoch %d: train loss = %f" % (epoch, running_loss))
    print(("          train acc = %f (%d/%d)" % (float(running_corrects/total_samples), running_corrects, total_samples)))

print("Training done in %f mins" % ((time.time()-start_time)/60))

Start to train


/home/mariapdg/hci/lib/python3.6/site-packages/ipykernel_launcher.py:20: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).


Epoch 0: train loss = 56000.014778
          train acc = 0.121400 (1214/10000)
Epoch 1: train loss = 56000.017849
          train acc = 0.127500 (1275/10000)
Epoch 2: train loss = 56000.015331
          train acc = 0.129900 (1299/10000)
Epoch 3: train loss = 56000.017864
          train acc = 0.124400 (1244/10000)
Epoch 4: train loss = 56000.017567
          train acc = 0.127700 (1277/10000)
Epoch 5: train loss = 56000.013710
          train acc = 0.121400 (1214/10000)
Epoch 6: train loss = 56000.012985
          train acc = 0.130400 (1304/10000)
Epoch 7: train loss = 56000.017784
          train acc = 0.126900 (1269/10000)
Epoch 8: train loss = 56000.014309
          train acc = 0.124900 (1249/10000)
Epoch 9: train loss = 56000.018353
          train acc = 0.124400 (1244/10000)
Training done in 21.051886 mins


In [14]:
print(preds)

tensor([6, 0, 6, 1, 0, 0, 6, 1])
